In [2]:
This jupyter notebook includes instructions about how to use `situational awareness` tool analyzing time series data.

SyntaxError: invalid syntax (<ipython-input-2-53533843055c>, line 1)

# Load necessary modules

In [51]:
import json
import csv
import pandas as pd
from isoweek import Week
from optimization import situational_awareness as sa
#reload(sa)
from optimization import problem
from optimization import filter_selection as fs

# Data format

Each data source is converted to the same format, and sotred in a list.
**NOTE**: change 

## A module to combine Data format, Preprocess data, and Optimization for situational awareness

- This module will use each csv file in `GoldStandard` as gold standard, and all time series in each csv file in `SourceToOptimize` as candidate data sources.

- It will write optimization results into new csv files and save in provided folder.

- Use this module for automated optimization process.

**NOTE**: All input csv files should have the same format.

In [33]:
import sklearn
sklearn.__version__

'0.21.2'

In [24]:
print(os.getcwd())
os.chdir(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
print(os.getcwd())

C:\Users\Remy\Desktop\UT Austin\Meyers Lab\Venezuela Situational Awareness\RemySituAwareness\optimization
C:\Users\Remy\Desktop\UT Austin\Meyers Lab\Venezuela Situational Awareness\RemySituAwareness


In [48]:
import os
import glob
from isoweek import Week
import pandas as pd

if os.getcwd()[-12:] != 'optimization':
    os.chdir(os.getcwd() + '\\optimization\\')
import problem
import filter_selection as fs
import situational_awareness as sa
os.chdir(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

def _to_json(_id, _times, _values):
    """
    Generate a dict
    """
    data_source = {'id': _id,
                   'data':
                   {'times': _times,
                    'values': _values}
                  }
    return data_source

def csv_to_json(file_path):
    """
    Convert data from csv format to Json format
    """
    # read csv
    csv_data = pd.read_csv(file_path)
    
    # check value in column 'year/week' is date or year/week
    if len(str(csv_data.loc[0, 'year/week']).split('/')) > 1:
        csv_data.loc[:, 'date'] = csv_data.loc[:, 'year/week'].apply(pd.to_datetime)
    else:
        # convert year /week to date
        csv_data.loc[:, 'year'] = [i[:4] for i in csv_data.loc[:, 'year/week'].apply(str)]
        csv_data.loc[:, 'week'] = [i[-2:] for i in csv_data.loc[:, 'year/week'].apply(str)]
        csv_data.loc[:, 'date'] = [pd.to_datetime(Week(int(csv_data.loc[i, 'year']),
                                        int(csv_data.loc[i, 'week'])).sunday()) for i in csv_data.index.values]
    
    # write into json
    column_names = csv_data.columns
    all_data = []
    for i in range(csv_data.shape[1]):
        if column_names[i] in ['year/week', 'year', 'week', 'date']:
            pass
        else:
            _id = column_names[i].replace('-', '_')
            _times = csv_data.loc[:, 'date'].tolist()
            _values = csv_data.loc[:, column_names[i]].tolist()
            data_source = {'id': _id,
                           'data':
                           {'times': _times,
                            'values': _values}
                          }
            all_data.append(data_source)
    return all_data


def filter_data(data,
                date_start='2010-11-22',
                date_end='2016-09-19'):
    """
    Filter data to make sure all data are within the same date range
    """
    for i in range(len(data)):
        bool_mask = fs.filter_on_times(data[i], after=date_start, before=date_end)
        d = fs.select(data[i], bool_mask)
        data[i] = d
    return data


def optimization(gold_standard,
                 candidate_data_sources,
               objective='R_squared',
               n_folds=15,
               output_size=141):
    """
    Run optimization for situational awareness.
    
    candidate_data_sources: list
    gold_standard: dict
    """
    # optimization
    sa_optimization = problem.FS_problem(gold_standard,
                                         candidate_data_sources,
                                         req_data=[],
                                         objective=objective,
                                         n_folds=n_folds)
    optimum, objective_value, objective_single = sa_optimization.optimize('forward_selection', choose=output_size)
    # predictions based on optimum
    prediction = sa.pred_CV(gold_standard, optimum)['data']
    return optimum, objective_value, objective_single, prediction


def main(gold_standard_folder,
         candidate_folder,
         date_start='2010-11-22',
         date_end='2016-09-19',
         objective='R_squared',
         n_folds=15,
         output_size=141,
         save_folder='OptimizationResults'):
    # retrieve all csv files in each folder
    gold_standard_paths = glob.glob(os.path.join(gold_standard_folder, '*'))
    candidate_paths = glob.glob(os.path.join(candidate_folder, '*'))
    
    for g_path in gold_standard_paths:
        # load gold standard
        print('Loading gold standard from {}'.format(g_path.split('/')[-1]))
        gold_standard = csv_to_json(g_path)
        print('Done loading gold standard.')
        
        for c_path in candidate_paths:
            # load each csv file in SourceToOptimize
            print('Loading candidate data sources from {}'.format(c_path.split('/')[-1]))
            candidate_data_sources = csv_to_json(c_path)
            print('Done loading candidate data sources.')
            
            # filter gold standard and candidate data sources
            gold_standard = filter_data(gold_standard,
                                        date_start=date_start,
                                        date_end=date_end)
            candidate_data_sources = filter_data(candidate_data_sources,
                                                 date_start=date_start,
                                                 date_end=date_end)
            print('Done filtering data.')
            
            # optimization
            optimum, objective_value, objective_single, prediction = optimization(gold_standard[0],
                                                                                  candidate_data_sources,
                                                                                  objective=objective,
                                                                                  n_folds=n_folds,
                                                                                  output_size=output_size)
            # get name of gold standard csv and candidate data csv names.
            g_name = g_path.split('/')[-1].split('.')[0]
            c_name = c_path.split('/')[-1].split('.')[0]
            
            # write to csv
            if not os.path.exists(save_folder):
                os.makedirs(save_folder)
                
            # save performance of each single data source in candidate
            ids = [d['id'] for d in candidate_data_sources]
            df = pd.DataFrame(columns=['id', 'objective_values'])
            df['id'] = ids
            df['objective_values'] = objective_single
            path = os.path.join(save_folder, '{}_{}_objective_single.csv'.format(g_name, c_name))
            df.to_csv(path)

            # save performance of optimal combination of data sources.
            id_optimum = [i['id'] for i in optimum]
            df = pd.DataFrame(columns=['id', 'objective_values'])
            df['id'] = id_optimum
            df['objective_values'] = objective_value[1:]
            path = os.path.join(save_folder, '{}_{}_objective_optimum.csv'.format(g_name, c_name))
            df.to_csv(path)

            # save prediction results
            df = pd.DataFrame(columns=['times', 'values'])
            df['times'] = prediction['times']
            df['values'] = prediction['values']
            path = os.path.join(save_folder, '{}_{}_prediction.csv'.format(g_name, c_name))
            df.to_csv(path)
                                
            print('Done optimization using gold standard from {} and candidate data from {}.\n'.format(g_path.split('/')[-1], c_path.split('/')[-1]))
    
    print('Done all optimizations!')

Run the following cell to do optimization.

It will detect each csv files in the `gold_standard_folder` and `candidate_folder`.

All optimization results (including optimized objective values, objective value for each single data source, out-of-sample prediction) will be saved to the `save_folder`.

**NOTE**: to run these functions successfully, all csv files should have exactly the SAME format.

In [49]:
main(gold_standard_folder='GoldStandard',
     candidate_folder='SourcesToOptimize',
      date_start='2005-01-02',
     date_end='2009-12-27',
#     date_start='2010-11-22',
#     date_end='2013-09-19',
     objective='R_squared',
     n_folds=6,
     output_size=10,
     save_folder='OptimizationResults')

Loading gold standard from GoldStandard\AMAZONAS-VE.csv
Done loading gold standard.
Loading candidate data sources from SourcesToOptimize\ColombiaPlusGT.csv
Done loading candidate data sources.
Done filtering data.


ValueError: Cannot have number of splits n_splits=6 greater than the number of samples: n_samples=0.

# Out-of-sample evaluation

Same logic as optimization.

- Need to put optimial combinations of predictor time series into one csv file, and put in a folder named 'SourcesForPrediction'

- Specify the start and end dates of training period AND testing period in the function 'main_oos'

- For each gold standard, the optimial combinations of predictor time series might be different. So each time running the experience, you should only put one gold standard in the 'GoldStandard' folder.

In [52]:
import os
import glob
from isoweek import Week
import pandas as pd

def oos_prediction(gold_standard,
                   predictor_data_sources,
                   train_period=['2004-01-01', '2006-01-01'],
                   test_period=['2008-01-01', '2009-01-01']):
    """
    Run out-of-sample evaluation for optimial models
    
    predictor_data_sources: list
    gold_standard: dict
    """
    # predictions based on optimum
    prediction = sa.pred_OOS(gold_standard,
                             predictor_data_sources,
                             train_period,
                             test_period)['data']
    return prediction


def oos_main(gold_standard_folder,
         predictor_folder,
         date_start='2010-11-22',
         date_end='2016-09-19',
         train_period=['2004-01-01', '2006-01-01'],
         test_period=['2008-01-01', '2009-01-01'],
         save_folder='OOSPredictionResults'):
    # retrieve all csv files in each folder
    gold_standard_paths = glob.glob(os.path.join(gold_standard_folder, '*'))
    predictor_paths = glob.glob(os.path.join(predictor_folder, '*'))
    
    for g_path in gold_standard_paths:
        # load gold standard
        print('Loading gold standard from {}'.format(g_path.split('/')[-1]))
        gold_standard = csv_to_json(g_path)
        print('Done loading gold standard.')
        for c_path in predictor_paths:
            # load each csv file in sourceForPrediction
            print('Loading predictor data sources from {}'.format(c_path.split('/')[-1]))
            predictor_data_sources = csv_to_json(c_path)
            print('Done loading predictor data sources.')
            
            # filter gold standard and predictor data sources
            gold_standard = filter_data(gold_standard,
                                        date_start=date_start,
                                        date_end=date_end)
            predictor_data_sources = filter_data(predictor_data_sources,
                                                 date_start=date_start,
                                                 date_end=date_end)
            print('Done filtering data.')
            
            # out-of-sample prediction
            prediction = oos_prediction(gold_standard[0],
                                        predictor_data_sources,
                                        train_period=train_period,
                                        test_period=test_period)

            # get name of gold standard csv and predictor data csv names.
            g_name = g_path.split('/')[-1].split('.')[0]
            c_name = c_path.split('/')[-1].split('.')[0]
            
            # write to csv
            if not os.path.exists(save_folder):
                os.makedirs(save_folder)

            # save oos prediction results
            df = pd.DataFrame(columns=['times', 'values'])
            df['times'] = prediction['times']
            df['values'] = prediction['values']
            path = os.path.join(save_folder, '{}_{}_OOSprediction.csv'.format(g_name, c_name))
            df.to_csv(path)
                                
            print('Done out-of-sample prediction using gold standard from {} and predictor data from {}.\n'.format(g_path.split('/')[-1], c_path.split('/')[-1]))
    
    print('Done all OOS prediction!')

In [53]:
oos_main(gold_standard_folder='GoldStandard',
     predictor_folder='SourcesForPrediction',
     date_start='2010-11-22',
     date_end='2013-09-19',
     train_period=['2010-11-22', '2012-09-19'],
     test_period=['2012-09-19', '2013-09-19'],
     save_folder='OOSPredictionResults')

Loading gold standard from GoldStandard\AMAZONAS-VE.csv
Done loading gold standard.
Loading predictor data sources from SourcesForPrediction\Colombia.csv
Done loading predictor data sources.
Done filtering data.


AttributeError: module 'optimization.situational_awareness' has no attribute 'pred_OOS'

# Plot

Plots should be generated based on the outputs. Two suggestions here:

- a time sereis plot showing gold standard and prediction.

- a plot showing how R^2 changes with the size of optimum.

You can also generate other plots or tables which you think can make sense.

# Goals of the analysis

There are two goals:

- identify what data sources are good predictors for dengue in Venezuela in national-level.

- identify what data sources are good predictors for dengue in Venezuela's states along the border.